In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup

In [2]:
mid_r = requests.get('http://artsongcentral.com/songs-for-medium-voice/')
mid_soup = BeautifulSoup(mid_r.text, 'html.parser')
mid_soup.prettify()

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html lang="en-US" xmlns="http://www.w3.org/1999/xhtml">\n <head>\n  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>\n  <title>\n   Songs for Medium Voice\n  </title>\n  <link href="http://artsongcentral.com/wp-content/themes/simply-works-core/style.css" media="screen" rel="stylesheet" type="text/css"/>\n  <link href="http://artsongcentral.com/xmlrpc.php" rel="pingback"/>\n  <link href="http://artsongcentral.com/wp-content/themes/simply-works-core/favicon.ico" rel="shortcut icon"/>\n  <link href="//s0.wp.com" rel="dns-prefetch"/>\n  <link href="//s.w.org" rel="dns-prefetch"/>\n  <link href="http://artsongcentral.com/feed/" rel="alternate" title="Art Song Central » Feed" type="application/rss+xml"/>\n  <link href="http://artsongcentral.com/comments/feed/" rel="alternate" title="Art Song Central » Comments Feed" type="application/rss+xml"/>\n  <

In [3]:
mid_a_tags = mid_soup.find_all('a')
mid_a_tags

[<a href="http://artsongcentral.com"> Art Song Central </a>,
 <a href="http://artsongcentral.com/song-index/">Song Index</a>,
 <a href="http://artsongcentral.com/source-index/">Source Index</a>,
 <a href="http://artsongcentral.com/songs-and-sources/">Browse By Type</a>,
 <a href="http://artsongcentral.com/songs-for-high-voice/">Songs for High Voice</a>,
 <a href="http://artsongcentral.com/songs-for-medium-voice/">Songs for Medium Voice</a>,
 <a href="http://artsongcentral.com/songs-for-low-voice/">Songs for Low Voice</a>,
 <a href="http://artsongcentral.com/songs-for-weddings/">Songs for Weddings</a>,
 <a href="http://artsongcentral.com/songs-for-christmas/">Songs for Christmas</a>,
 <a href="http://artsongcentral.com/songs-in-german/">Songs in German</a>,
 <a href="http://artsongcentral.com/songs-in-spanish/">Songs in Spanish</a>,
 <a href="http://artsongcentral.com/songs-in-english/">Songs in English</a>,
 <a href="http://artsongcentral.com/songs-in-latin/">Songs in Latin</a>,
 <a hr

In [4]:
mid_div_tags = mid_soup.find('div', class_ = 'entry')
mid_div_tags

<div class="entry">
<p>This is an index of all song posts that have been tagged as being appropriate for medium voices (Mezzo or Baritone). You may wish to consult the lists for high and low voices, as well, since there is no clear defining line for what belongs in each category.</p>
<p>Also, there may be songs on this site which exist elsewhere in a medium key, or are available in a format that allows for transposition. So, if you are looking for something specific, <strong>please use the search function</strong> as well! (And if you still can’t find what you’re looking for here, you may be able to purchase it at <a href="http://www.sheetmusicplus.com/a/home.html?id=217645" target="_blank">Sheet Music Plus</a> and simultaneously provide support for this site.)</p>
<p><ul><li><a href="http://artsongcentral.com/2007/adam-cantique-de-noel/">Adam - Cantique de Noël</a></li><li><a href="http://artsongcentral.com/2009/beach-hymn-of-trust/">Beach - Hymn of Trust</a></li><li><a href="http://a

In [5]:
mid_songs = []

for a in mid_div_tags.findAll('a'):
    mid_songs.append(a.text)


mid_links = []
      
for a in mid_a_tags:
    mid_links.append(a['href'])

In [6]:
mid_links_cleaned = mid_links[17:236]
mid_songs_cleaned = mid_songs[1:]
mid_df = pd.DataFrame(np.column_stack([mid_songs_cleaned, mid_links_cleaned]), 
                               columns=['song', 'link'])
composer = mid_df.song.str.split('-').str.get(0)
title = mid_df.song.str.split('-').str.get(1)
mid_df['composer'] = composer
mid_df['title'] = title
columnsTitles=["composer", "title","link"]
mid_df = mid_df.reindex(columns=columnsTitles)
print(mid_df.head())
print(mid_df.tail())

     composer                              title  \
0       Adam                    Cantique de Noël   
1      Beach                       Hymn of Trust   
2      Beach                       The Rainy Day   
3      Beach            The Year's at the Spring   
4  Beethoven    Ich liebe Dich (Zärtliche Liebe)   

                                                link  
0  http://artsongcentral.com/2007/adam-cantique-d...  
1  http://artsongcentral.com/2009/beach-hymn-of-t...  
2  http://artsongcentral.com/2009/beach-the-rainy...  
3  http://artsongcentral.com/2009/beach-the-years...  
4  http://artsongcentral.com/2007/beethoven-ich-l...  
     composer                 title  \
214     Wolf             Epiphanias   
215     Wolf              Fussreise   
216     Wolf         Gesang Weyla's   
217     Wolf    Verschwiegene Liebe   
218  Yradier              La Paloma   

                                                  link  
214    http://artsongcentral.com/2008/wolf-epiphanias/  
215     

In [7]:
mid_df.to_json(path_or_buf='mid_songs.json', orient='records')

In [8]:
df_counts = mid_df.groupby(['composer'], as_index=False).count()
df_counts.head()

,composer,title,link
0,Adam,1,1
1,Beach,3,3
2,Beethoven,2,2
3,Bemberg,1,1
4,Berlioz,1,1


In [9]:
df_counts.describe()

,title,link
count,88.000000,88.000000
mean,2.488636,2.488636
std,2.555114,2.555114
min,1.000000,1.000000
25%,1.000000,1.000000
50%,1.000000,1.000000
75%,3.000000,3.000000
max,13.000000,13.000000


In [10]:
print(df_counts.sort_values(by=['title'], ascending=False).head(10))
print(df_counts.sort_values(by=['title'], ascending=False).tail(20))

             composer  title  link
81  Vaughan Williams      13    13
26           Debussy      12    12
35             Fauré      10    10
73         Scarlatti       9     9
21          Chausson       8     8
30           Dowland       7     7
68           Quilter       7     7
87          de Falla       7     7
83              Wolf       7     7
74          Schubert       6     6
       composer  title  link
33  Falconieri       1     1
34    Faure, J       1     1
37      Franck       1     1
38       Franz       1     1
39    Giordani       1     1
41     Griffes       1     1
43        Hahn       1     1
58  Monteverdi       1     1
45       Haydn       1     1
46      Holmès       1     1
48      Jensen       1     1
49    Jommelli       1     1
50       Jones       1     1
51        Lang       1     1
52       Lawes       1     1
53    Legrenzi       1     1
22       Ching       1     1
55     Machaut       1     1
57     Martini       1     1
0         Adam       1     1


In [11]:
import matplotlib.pyplot as plt
plt.hist(df_counts.title, bins=12)

(array([47., 17.,  9.,  1.,  3.,  2.,  4.,  1.,  1.,  1.,  0.,  2.]),
 array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.]),
 <a list of 12 Patch objects>)

While this is not a large dataset and is not terribly representative of composers and art songs, it is a decent start. Near the top of the list are composers like Schubert, Debussy, and Faure, who were among the most prolific art song composers. At the very top of this list is Ralph Vaughan Williams, who didn't write a ton of art songs, so I believe that this may be because of bias introduced by the website's author(s) and their proclivity towards Vaughan Williams' music. Towards the tail end of the count frequency list, we see familiar names such as Haydn, Beethoven, and Monteverdi, who despite being major composers in their owns rights, did not produce a great deal of art songs. 

In the histogram, we can observe that there are nearly 50 composers (out of the 88 here) who only composed one art song as represented in this data set. There are a god handful of composers who composed two or three. Higher frequencies sparsley populate the remainder of this right-skewed distribution. 